In [1]:
# 层和块

## 块由类表示，任何子类都必须定义一个将其输入转换为输出的前向传播函数

In [2]:
import torch
from torch import nn
from torch.nn import functional as F  # 一组函数式的API，它提供了许多神经网络操作的函数实现（如激活函数、卷积操作、池化操作等）

In [ ]:
net = nn.Sequential(nn.Linear(20, 256),  nn.ReLU(), nn.Linear(256, 10))  
        # Again, nn.Sequential是PyTorch的一个容器，用于按顺序将多个层组合在一起

X = torch.rand(2, 20)
net(X)  # net.__call__(X)的简写

tensor([[ 0.3255,  0.0831, -0.0611,  0.1000, -0.2144, -0.0971,  0.0024, -0.0198,
          0.1175, -0.0229],
        [ 0.1743,  0.0020, -0.1273,  0.2158, -0.0590, -0.1780,  0.0953, -0.1102,
          0.1143, -0.0151]], grad_fn=<AddmmBackward0>)

In [5]:
# 自定义块
# 每个块必须提供的基本功能：
# 1. 将输入数据作为其前向传播的参数
# 2. 通过前向传播函数来生成输出
# 3. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。（通常自动发生）
# 4. 存储和访问前向传播计算所需的参数
# 5. 根据需要初始化模型参数

In [7]:
class MLP(nn.Module):
    # 用模型参数声明层。这里我们声明两个全连接的层
    def __init__(self):
        # 调用父类Module的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    # 定义模型的前向传播
    def forward(self, X):
        # 这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

In [9]:
net = MLP()
net(X)

tensor([[ 0.1441,  0.0060, -0.0849, -0.1101,  0.0168,  0.1831,  0.0741,  0.0826,
          0.0683,  0.0327],
        [ 0.0600,  0.0452, -0.1666, -0.0795,  0.0799,  0.2856,  0.2310,  0.1618,
         -0.0885,  0.0475]], grad_fn=<AddmmBackward0>)

In [12]:
# 顺序块Sequential
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):  # 用enumerate遍历传入的args元组，enumerate()会同时返回当前元素的索引和元素值
            # 这里，module时Module子类的一个实例，我们把它保存在'Module'类的实例成员变量_modules中。__modules的类型是OrderedDict
            # 成员变量是绑定到某个具体对象（实例）的变量，即仅作用于该对象（实例）本身
            # _modules中，单下划线前缀是一种约定，表示该变量或方法是“受保护的”或“内部使用的”
            self._modules[str(idx)] = module
        
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X


In [ ]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)  # nn.Module重载了__call__方法，会在调用net(X)时自动调用子类中定义的forward方法

tensor([[ 0.0310, -0.1787, -0.0285, -0.0208,  0.1479, -0.0316, -0.0357, -0.0278,
          0.1241, -0.0650],
        [-0.0404, -0.1686,  0.2224, -0.2464,  0.1851, -0.0212, -0.1189, -0.0934,
          0.0141,  0.1771]], grad_fn=<AddmmBackward0>)

In [14]:
# 在前向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()




In [15]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0814, grad_fn=<SumBackward0>)

In [17]:
# 混合搭配各种组合块
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0421, grad_fn=<SumBackward0>)